# Experiment tracking 

In [2]:
import mlflow

mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("check-localhost-connection")

with mlflow.start_run():
    mlflow.log_metric("foo", 1)
    mlflow.log_metric("bar", 2)

2024/09/05 14:27:59 INFO mlflow.tracking._tracking_service.client: 🏃 View run sincere-loon-767 at: http://localhost:5000/#/experiments/745768468464273167/runs/e5f61c7a0a164c72b5bcac6cdce59100.
2024/09/05 14:27:59 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/745768468464273167.


In [15]:
from sklearn.datasets import make_regression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

import mlflow
import mlflow.sklearn

with mlflow.start_run() as run:
    X, y = make_regression(n_features=4, n_informative=2, random_state=0, shuffle=False)
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    params = {"max_depth": 2, "random_state": 42}
    model = RandomForestRegressor(**params)
    model.fit(X_train, y_train)
    mlflow.set_tag("devloper","Venu")
    # Log parameters and metrics using the MLflow APIs
    mlflow.log_params(params)

    y_pred = model.predict(X_test)
    mlflow.log_metrics({"mse": mean_squared_error(y_test, y_pred)})

    # Log the sklearn model and register as version 1
    mlflow.sklearn.log_model(
        sk_model=model,
        artifact_path="sklearn-model",
        input_example=X_train,
        registered_model_name="sk-learn-random-forest-reg-model",
        
    )
    
    # mlflow.set_experiment_tags("test")


Registered model 'sk-learn-random-forest-reg-model' already exists. Creating a new version of this model...
2024/09/05 14:54:47 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: sk-learn-random-forest-reg-model, version 4
Created version '4' of model 'sk-learn-random-forest-reg-model'.
2024/09/05 14:54:52 INFO mlflow.tracking._tracking_service.client: 🏃 View run adaptable-stork-524 at: http://localhost:5000/#/experiments/745768468464273167/runs/131af82465c54b91a33565cc844faf78.
2024/09/05 14:54:52 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/745768468464273167.


In [4]:
# adding the tags 

In [5]:
from mlflow import MlflowClient

# Initialize an MLflow Client
client = MlflowClient()

In [7]:
def assign_alias_to_stage(model_name, stage, alias):
    """
    Assign an alias to the latest version of a registered model within a specified stage.

    :param model_name: The name of the registered model.
    :param stage: The stage of the model version for which the alias is to be assigned. Can be
                "Production", "Staging", "Archived", or "None".
    :param alias: The alias to assign to the model version.
    :return: None
    """
    latest_mv = client.get_latest_versions(model_name, stages=[stage])[0]
    client.set_registered_model_alias(model_name, alias, latest_mv.version)

C:\Users\venu_reddy\AppData\Local\Temp\ipykernel_42852\2930693847.py:4: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest_mv = client.get_latest_versions(model, stages=[stage])[0]


IndexError: list index out of range